In [0]:
def extract_data(url, fname=None, dest=None, force_download=False):
    from urllib.request import urlopen
    from zipfile import ZipFile
    import os
    import shutil
    from pathlib import Path
    filename = url.rpartition('/')[2]
    foldername = filename.split('.')[0]
    if os.path.exists(f'{dest}/{foldername}'):
        print('Already Downloaded')
        if force_download:
            print('Removing Old Downloaded')
            shutil.rmtree(f'{dest}/{foldername}')
        else:
            return Path(f'{dest}/{foldername}')
    print('Downloading from remote server')
    response = urlopen(url)
    if dest is None: dest = '.'
    tmpzip = open(f'{dest}/{filename}', 'wb')
    tmpzip.write(response.read())
    tmpzip.close()
    if (filename.endswith("zip")):
        zf = ZipFile(f'{dest}/{filename}')
        zf.extractall(path = dest)
        zf.close()
    elif (filename.endswith("tar.gz") or filename.endswith("tgz")):
        tar = tarfile.open(filename, "r:gz")
        tar.extractall(path = dest)
        tar.close()
    elif (filename.endswith("tar")):
        tar = tarfile.open(filename, "r:")
        tar.extractall(path = dest)
        tar.close()
    os.remove(f'{dest}/{filename}')
    return Path(f'{dest}/{foldername}')

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models, transforms, datasets
import matplotlib.pyplot as plt
import time
import os
import copy
from pathlib import Path
from PIL import Image
import numpy as np

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [29]:
device

device(type='cuda', index=0)

In [4]:
path = extract_data('http://files.fast.ai/data/dogscats.zip', dest='sample_data')
list(iter(path.iterdir()))

[PosixPath('sample_data/dogscats/models'),
 PosixPath('sample_data/dogscats/valid'),
 PosixPath('sample_data/dogscats/train'),
 PosixPath('sample_data/dogscats/sample'),
 PosixPath('sample_data/dogscats/test1')]

In [0]:
path_train = path/'train'
path_valid = path/'valid'
path_test  = path/'test1'

In [0]:
size=224
batch = 64 

In [0]:
imagenet_stats = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

In [0]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(*imagenet_stats),
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size),
        transforms.CenterCrop(size),
        transforms.ToTensor(),
        transforms.Normalize(*imagenet_stats)
    ]),
    'test': transforms.Compose([
        transforms.Resize(size),
        transforms.CenterCrop(size),
        transforms.ToTensor(),
        transforms.Normalize(*imagenet_stats)
    ])
}

In [0]:
from skimage.io import imread

In [0]:
class CustomImageDatset(Dataset):
    def __init__(self, root_dir, train, transform=None,  mode='train'):
        self.root_dir = root_dir
        self.train = train
        self.filenames = []
        self.targets = []
        self.transform = transform
        
        self.classes = os.listdir(f'{root_dir}/{train}')
        self.c = len(self.classes)
        for i, folder in enumerate(self.classes):
            for name in os.listdir(f'{root_dir}/{train}/{folder}'):
                self.filenames.append(f'{root_dir}/{train}/{folder}/{name}')
                self.targets.append(i)
        self.len = len(self.filenames)

    def __len__(self):
        return self.len
    
    def __getitem__(self, index):
#         import pdb; pdb.set_trace();
        img_name = self.filenames[index]
        target = self.targets[index]
        image = Image.open(img_name)
        if self.transform:
            image = self.transform(image)
        image = image / 255
        data = (image, target)
        
        return data

In [0]:
train_datasets = CustomImageDatset(path, 'train', data_transforms['train'])
valid_datasets = CustomImageDatset(path, 'valid', data_transforms['valid'])

In [0]:
train_dataloader = DataLoader(train_datasets, batch_size=batch, shuffle=True)
valid_dataloader = DataLoader(valid_datasets, batch_size=batch, shuffle=False)

In [0]:
loaders = {
    'train': train_dataloader,
    'valid': valid_dataloader
}

In [0]:
# train_data = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
# valid_data = datasets.ImageFolder(valid_dir, transform=data_transforms['val'])
# test_data = datasets.ImageFolder(test_dir, transform=data_transforms['test'])

In [14]:
arch = models.resnet34(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.torch/models/resnet34-333f7ec4.pth
87306240it [00:01, 49279970.07it/s]


In [0]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)


In [0]:
# arch.fc = nn.Sequential(
#   nn.AdaptiveAvgPool2d(output_size=1),
#   nn.AdaptiveMaxPool2d(output_size=1),
#   Flatten(),
#   nn.BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#   nn.Dropout(p=0.25),
#   nn.Linear(in_features=1024, out_features=512, bias=True),
#   nn.ReLU(),
#   nn.BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#   nn.Dropout(p=0.5),
#   nn.Linear(in_features=512, out_features=train_datasets.c, bias=True),
# )

arch.fc = nn.Linear(512, 2, bias=True)

In [0]:
arch

In [0]:
arch = arch.cuda()

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(arch.parameters(), lr=1e-2, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [0]:
import time

In [0]:
# def train(epochs, loader, model, optimizer, criterion, save_path):
#     since = time.time()
#     val_acc_history = []
#     best_model_wts = copy.deepcopy(model.state_dict())
#     best_acc = 0.0
    
#     for epoch in range(1, epochs + 1):
#         print('Epoch {}/{}'.format(epoch, epochs))
#         print('-' * 10)
        
#         for phase in ['train', 'valid']:
#             if phase == 'train':
#                 model.train()
#             else:
#                 model.eval()
                
#             running_loss = 0.0
#             running_corrects = 0
#             for batch_idx, (data, target) in enumerate(loaders[phase]):
#                 data, target = data.cuda(), target.cuda()
#                 optimizer.zero_grad()
                
#                 with torch.set_grad_enabled(phase == 'train'):
#                     outputs = model(data)
#                     loss = criterion(outputs, target)
#                     _, preds = torch.max(outputs, 1)
#                 if phase == 'train':
#                     loss.backward()
#                     optimizer.step()

#             # statistics
#             running_loss += loss.item() * data.size(0)
#             running_corrects += torch.sum(preds == target.data)            
#         epoch_loss = running_loss / len(loaders[phase].dataset)
#         epoch_acc = running_corrects.double() / len(loaders[phase].dataset)

#         print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
#         # deep copy the model
#         if phase == 'val' and epoch_acc > best_acc:
#             best_acc = epoch_acc
#             best_model_wts = copy.deepcopy(model.state_dict())
#         if phase == 'val':
#             val_acc_history.append(epoch_acc)
#     print()
#     time_elapsed = time.time() - since
#     print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
#     print('Best val Acc: {:4f}'.format(best_acc))

#     # load best model weights
#     model.load_state_dict(best_model_wts)
#     return model, val_acc_history



In [0]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False


In [0]:
feature_extract = True

In [0]:
params_to_update = arch.parameters()

if feature_extract:
    params_to_update = []
    for name,param in arch.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
#             print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            pass
#             print("\t",name)

In [0]:
set_parameter_requires_grad(arch, feature_extract)

In [0]:
for param in arch.fc.parameters():
    param.requires_grad = True

In [0]:
def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(loaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(loaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [105]:
model = train_model(arch, loaders, criterion, optimizer, exp_lr_scheduler, num_epochs=3)

Epoch 0/2
----------
train Loss: 0.1225 Acc: 0.9493
valid Loss: 0.0301 Acc: 0.9875

Epoch 1/2
----------
train Loss: 0.1221 Acc: 0.9483
valid Loss: 0.0300 Acc: 0.9865

Epoch 2/2
----------
train Loss: 0.1145 Acc: 0.9507
valid Loss: 0.0303 Acc: 0.9865

Training complete in 8m 47s
Best val Acc: 0.000000
